# Job Reassigment Problem with QAOA:
## Small instances and GPU running

In this notebook, we will experiment with the small instances generated by "random_instances_generation.ipynb". A simulation for QAOA optimization will be done for each instance. First, using a personal notebook CPU. Then, using google colab GPU.

The goals of the notebooks are:
* trying QAOA with small JRP instances that follows the heuristic rules (mentione in "random_instances_generation.ipynb").
* Achieving, at least, regular bistring measurements distributions for the QAOA optimizations. That is, not focusing on having excelent results, but that we are going in a good path.
* Comparing the simulation speedup between the personal notebook CPU vs. google colab GPU. This will be helpful for future experiments we will be working with bigger and more important instances.

For running the experiments, using a local envirioment and the google colab will be necessary.

## For Google Colab

The GPU experiment part will make use of Google Colab. Please only only follow the next instructions in case you are going to use Google Colab $\textbf{NOW}$. Doing the following in your local envirioment will install in your own PC openqaoa an other libraries in a local mode, which is not recomended. If you are going to keep using the local envirioment until the GPU experiment requires you to migrate to Google Colab, jump to the next section and came back here when the instructions tells you.

In [1]:
# 1. RUN THIS
!git clone https://github.com/entropicalabs/openqaoa.git

Cloning into 'openqaoa'...
remote: Enumerating objects: 12606, done.
remote: Counting objects: 100% (2969/2969), done.
remote: Compressing objects: 100% (736/736), done.
remote: Total 12606 (delta 2498), reused 2473 (delta 2221), pack-reused 9637
Receiving objects: 100% (12606/12606), 22.03 MiB | 14.20 MiB/s, done.
Resolving deltas: 100% (9365/9365), done.


In [2]:
# 2. RUN THIS
!git clone https://github.com/AdrianoLusso/Tesis.git

Cloning into 'Tesis'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 77 (delta 22), reused 71 (delta 16), pack-reused 0
Receiving objects: 100% (77/77), 28.00 MiB | 9.17 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
# 3. RUN THIS
with open('Tesis/openqoa_modifiedFiles/Makefile.txt', 'r') as file:
    replacement_content = file.read()

with open('openqaoa/Makefile', 'w') as file:
    file.write(replacement_content)

#verify a successful writing
!cat openqaoa/Makefile

In [ ]:
# 4. RUN THIS
with open('Tesis/openqoa_modifiedFiles/requirements_core.txt', 'r') as file:
    replacement_content = file.read()

with open('openqaoa/src/openqaoa-core/requirements.txt', 'w') as file:
    file.write(replacement_content)

#verify a successful writing
!cat openqaoa/src/openqaoa-core/requirements.txt

In [ ]:
# 5. RUN THIS
with open('Tesis/openqoa_modifiedFiles/requirements_qiskit.txt', 'r') as file:
    replacement_content = file.read()

with open('openqaoa/src/openqaoa-qiskit/requirements.txt', 'w') as file:
    file.write(replacement_content)

#verify a successful writing
!cat openqaoa/src/openqaoa-qiskit/requirements.txt

In [ ]:
# 6. RUN THIS
with open('Tesis/openqoa_modifiedFiles/qaoa_qiskit_sim.txt', 'r') as file:
    replacement_content = file.read()

with open('openqaoa/src/openqaoa-qiskit/openqaoa_qiskit/backends/qaoa_qiskit_sim.py', 'w') as file:
    file.write(replacement_content)

#verify a successful writing
!cat openqaoa/src/openqaoa-qiskit/openqaoa_qiskit/backends/qaoa_qiskit_sim.py

In [ ]:
#7. RUN THIS
!cd openqaoa && make local-install

In [ ]:
#8. RUN THIS
!pip install qiskit-optimization

In [13]:
# 9. RUN THIS
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    experiment_path = './Tesis/experiment_smallInstances/'
else:
    experiment_path = './'

## Importing the problem instances

In [14]:
import json
from openqaoa import QUBO
from openqaoa.backends import create_device
from openqaoa import QAOA
from openqaoa.algorithms import QAOAResult

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer

import timeit

In [15]:
instances_indexes = range(5)
instances = []
for index in instances_indexes:
     with open(experiment_path+'instance%s/instance%s.json'%(str(index),str(index)), "r") as file:
         instances.append(json.load(file))
#instances[0]

In [16]:
keys = list(instances[0].keys())
#keys

## Creating the QUBO formulation

In [17]:
if IN_COLAB:
  %run ./Tesis/functions/qubo_for_jrp.ipynb
  %run  ./Tesis/functions/qubo_in_openqaoa_format.ipynb
else:
  %run ../functions/qubo_for_jrp.ipynb
  %run ../functions/qubo_in_openqaoa_format.ipynb

In [18]:
qubos = []
qubos_openqaoaformat = []
for inst in instances:
    qubo = qubo_for_jrp(inst)
    qubos.append(qubo)
    qubo = qubo_in_openqaoa_format(qubo)
    qubos_openqaoaformat.append(qubo)
#qubos[0]
#qubos_openqaoaformat[0]

## Creating the Ising formulation for OpenQAOA

In [19]:
isings = []
for index,qubo in enumerate(qubos_openqaoaformat):
    ising= QUBO.convert_qubo_to_ising(len(instances[index]['allBinaryVariables']), qubo[0], qubo[1])
    isings.append(ising)
#isings[0]

Please notice that, in OpenQAOA library, it is called to a QUBO problem to a $\textbf{ising formulation}$ of the problem. That means, with the binary variables domain $\{-1,1\}$. In OpenQAOA documentation, this is because they consider it as a $\textbf{QUBO formulation with an Ising encryption}$.

In [20]:
openqaoa_qubos = []
for index,ising in enumerate(isings):
    openqaoa_qubo = QUBO(n = len(instances[index]['allBinaryVariables']), terms=ising[0], weights=ising[1])
    openqaoa_qubos.append(openqaoa_qubo)

## Using CplexOptimizer from Qiskit Optimization for classical solving

Explicar un poco de qiskit optimization y cplex

In [ ]:
print('----------------------------------------------------------------------------------------------')
for index,qubo in enumerate(qubos):
    num_variables = instances[index]['num_agents'] * instances[index]['num_vacnJobs']

    linear = {clave: qubo[clave] for clave in list(qubo.keys())[:num_variables]}
    quad = {clave: qubo[clave] for clave in list(qubo.keys())[num_variables:]}
    #print(linear)
    #print(quad)

    mod = QuadraticProgram("JRP")
    for i in  instances[index]['allBinaryVariables']:
        mod.binary_var(name="x"+str(i))

    mod.minimize(constant=3, linear=linear, quadratic=quad)
    #print(mod.prettyprint())
    result = CplexOptimizer().solve(mod)

    print('INSTANCE ',index,'\n')
    print(result.prettyprint())

    resultstring =""
    for val in result.variables_dict.values():
        resultstring += str(int(val))

    print('Solution : ',resultstring,'\n')
    print('----------------------------------------------------------------------------------------------')

## Using OpenQAOA brute force solver

OpenQAOA recommend bounding the brute force solver for <25 qubits problems.  Instance 4 has 30 variables. That's why we only search with this solver the solutions of the first four instances. The solutions given by the code below this markdown were:

* instance 0: Ground State energy: -3.9200000000000017, Solution: ['000000000001000']

* instance 1: Ground State energy: -3.9399999999999977, Solution: ['000001010000000000']

* instance 2: Ground State energy: -5.150000000000006, Solution: ['00000000010010000000']

* instance 3: Ground State energy: -2.8200000000000074, Solution: ['0000001000000001000000000']

In [ ]:
for openqaoa_qubo in openqaoa_qubos:
    solver = QAOA()
    solver.compile(openqaoa_qubo)
    solver.solve_brute_force(verbose=True)
    print()

## Running QAOA - personal notebook CPU

For running in a persona notebook CPU, we will limit to just using the first 3 instances, which also are the smallest ones. Instance 3 and 4 need 25 and 30 qubits respectively. These would result in a too big execution time, which goes out the aim of this notebook.

In [ ]:
#QAOA results and optimization execution times for each pair (instance,maxfev)
maxfevs = [1,2,3,4,5]
for index,qubo in enumerate(openqaoa_qubos[0:3]):
    execution_result={}
    for maxfev in maxfevs:
        execution_result['maxfev '+str(maxfev)] = {}

        q = QAOA()
        device = create_device(location='local', name='qiskit.shot_simulator')
        q.set_device(device)
        q.set_circuit_properties(p=15, param_type='standard', init_type='ramp', mixer_hamiltonian='x')
        q.set_backend_properties(n_shots=10000,prepend_state=None, append_state=None)
        q.set_classical_optimizer(method='nelder-mead', maxiter=500, tol=0.001,maxfev=maxfev,
                          optimization_progress=False, cost_progress=True, parameter_log=False)
        q.compile(qubo)

        start_time = timeit.default_timer()
        q.optimize()
        execution_time = timeit.default_timer() - start_time
        print('exe time for instance ',index,' and maxfev ',maxfev,': ',execution_time)

        execution_result['maxfev '+str(maxfev)]['time'] = execution_time
        execution_result['maxfev '+str(maxfev)]['result'] = q.result.asdict()
    with open('instance%s/personalNotebook.json'%(str(index)), 'w') as file:
        json.dump(execution_result, file)

## Running QAOA - personal notebook CPU

In this moment is when you need to migrate to Google Colab. At the beggining of the notebook, there was a section for a proccess to do for a successful migration. Please, open this notebook in Google Colab, go back to that section and run it all.

After that, also run again the sections 'Importing the problem instances', 'Creating the QUBO formulation' and 'Creating the Ising formulation for OpenQAOA'. Then, you come back here and keep running the experiment.

In [ ]:
#QAOA results and optimization execution times for each pair (instance,maxfev)
maxfevs = [100,2,3,4,5]
for index,qubo in enumerate(openqaoa_qubos[4:5]):
    execution_result={}
    for maxfev in maxfevs:
        execution_result['maxfev '+str(maxfev)] = {}

        q = QAOA()
        device = create_device(location='local', name='qiskit.shot_simulator')
        q.set_device(device)
        q.set_circuit_properties(p=15, param_type='standard', init_type='ramp', mixer_hamiltonian='x')
        q.set_backend_properties(n_shots=10000,prepend_state=None, append_state=None)
        q.set_classical_optimizer(method='nelder-mead', maxiter=500, tol=0.001,maxfev=maxfev,
                          optimization_progress=False, cost_progress=True, parameter_log=False)
        q.compile(qubo)
        start_time = timeit.default_timer()
        q.optimize()
        execution_time = timeit.default_timer() - start_time
        print('exe time for instance ',index,' and maxfev ',maxfev,': ',execution_time)

        execution_result['maxfev '+str(maxfev)]['time'] = execution_time
        execution_result['maxfev '+str(maxfev)]['result'] = q.result.asdict()
    with open('Tesis/experiment_smallInstances/instance%s/personalNotebook.json'%(str(index)), 'w') as file:
        json.dump(execution_result, file)